In [ ]:
import configparser as cp
import requests
from openai import OpenAI

def signature(channel_secret):
    import base64
    import hashlib
    import hmac
    body = '...' # Request body string
    hash = hmac.new(channel_secret.encode('utf-8'),
        body.encode('utf-8'), hashlib.sha256).digest()
    return base64.b64encode(hash)
    # Compare x-line-signature request header and the signature

from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage


config = cp.ConfigParser()
config.read('config.ini')

channel_secret = config['Line']['channel_secret']



def Get_user(Res):
    return Res['event'][0]['source']['userId']

def Get_replyToken(Res):
    return Res['event']['replyToken']

def ChatGPT(Message):
    Client = OpenAI(api_key=config['ChatGPT']['secret_key'])
    try:
        completion = Client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "你的主人叫做施博剴，而你是個默認使用繁體中文的有趣助理，偶爾會對使用者開玩笑。"},
                {"role": "user", "content": Message}
            ]
        )
        return completion.choices[0].message.content
    except Exception:
        return "或許主人的chatGPT沒有餘額了!!"   

def reply(messageText, replyToken):
    api = "https://api.line.me/v2/bot/message/reply"
    Header = {"Content-Type":"application/json",
              "Authorization": "Bearer {}".format(config['Line']['channel_access_token'])
    }
    Data = {"replyToken":replyToken,
            "messages":[
                {
                    "type":"text",
                    "text":messageText
                }
            ]
    }
    try:
        return requests.post(url=api, header=Header, data=Data)
    except Exception:
        return False



In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)
import configparser as cp
from openai import OpenAI

app = Flask(__name__)

configuration = Configuration(access_token='YOUR_CHANNEL_ACCESS_TOKEN')
handler = WebhookHandler('YOUR_CHANNEL_SECRET')


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text)]
            )
        )

if __name__ == "__main__":
    app.run()